In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
import nibabel as nib
import os
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm_notebook as tqdm

In [ ]:
from dice_score import *
from knee_init import *
from New_network import *

1.7.0+cu101


In [ ]:
class UNet_2d(nn.Module):
  def __init__(self, input_channels =1, num_classes = 2):
    super(UNet_2d, self).__init__()
    self.input_channels = input_channels
    
    ####DOWNSAMPLING####
    self.conv1 = Downconv(1, 64)
    self.maxpool1 = nn.MaxPool2d(kernel_size =2)
    self.conv2 = Downconv(64,128)
    self.maxpool2 = nn.MaxPool2d(kernel_size =2)
    self.conv3 = Downconv(128,256)
    self.maxpool3 = nn.MaxPool2d(kernel_size =2)
    self.conv4 = Downconv(256,512)
    self.maxpool4 = nn.MaxPool2d(kernel_size =2)

    self.conv5 = Downconv(512,1024)


    ####UPSAMPLINIG####
    self.upconv4 = Upconv(1024,512)
    self.upconv3 = Upconv(512,256)
    self.upconv2 = Upconv(256,128)
    self.upconv1 = Upconv(128,64)

    ####Final/Output####

    self.output = output(64,num_classes)

  def forward(self, inputs):
     
    
    conv1 = self.conv1(inputs)
    maxpool1 = self.maxpool1(conv1)

    conv2 = self.conv2(maxpool1)
    maxpool2 = self.maxpool2(conv2)

    conv3 = self.conv3(maxpool2)
    maxpool3 = self.maxpool3(conv3)

    conv4 = self.conv4(maxpool3)
    maxpool4 = self.maxpool4(conv4)

    center = self.conv5(maxpool4)    # final convolution of encoderpart without maxpool layer
      
    up4 = self.upconv4(center, conv4)
    up3 = self.upconv3(up4, conv3)
    up2 = self.upconv2(up3, conv2)
    up1 = self.upconv1(up2, conv1)

    output = self.output(up1)      # output layer/ final layer 

    return output
  @staticmethod
  def apply_argmax_softmax(pred):
    log_p = F.softmax(pred, dim=1)

    return log_p
class UNet_2d(nn.Module):
  def __init__(self, input_channels =1, num_classes = 2):
    super(UNet_2d, self).__init__()
    self.input_channels = input_channels
    
    ####DOWNSAMPLING####
    self.conv1 = Downconv(1, 64)
    self.maxpool1 = nn.MaxPool2d(kernel_size =2)
    self.conv2 = Downconv(64,128)
    self.maxpool2 = nn.MaxPool2d(kernel_size =2)
    self.conv3 = Downconv(128,256)
    self.maxpool3 = nn.MaxPool2d(kernel_size =2)
    self.conv4 = Downconv(256,512)
    self.maxpool4 = nn.MaxPool2d(kernel_size =2)

    self.conv5 = Downconv(512,1024)


    ####UPSAMPLINIG####
    self.upconv4 = Upconv(1024,512)
    self.upconv3 = Upconv(512,256)
    self.upconv2 = Upconv(256,128)
    self.upconv1 = Upconv(128,64)

    ####Final/Output####

    self.output = output(64,num_classes)

  def forward(self, inputs):
     
    
    conv1 = self.conv1(inputs)
    maxpool1 = self.maxpool1(conv1)

    conv2 = self.conv2(maxpool1)
    maxpool2 = self.maxpool2(conv2)

    conv3 = self.conv3(maxpool2)
    maxpool3 = self.maxpool3(conv3)

    conv4 = self.conv4(maxpool3)
    maxpool4 = self.maxpool4(conv4)

    center = self.conv5(maxpool4)    # final convolution of encoderpart without maxpool layer
      
    up4 = self.upconv4(center, conv4)
    up3 = self.upconv3(up4, conv3)
    up2 = self.upconv2(up3, conv2)
    up1 = self.upconv1(up2, conv1)

    output = self.output(up1)      # output layer/ final layer 

    return output
  @staticmethod
  def apply_argmax_softmax(pred):
    log_p = F.softmax(pred, dim=1)

    return log_p


In [ ]:
def init_weights(m):   # weights initialized using xavier
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        nn.init.xavier_normal(m.weight)
        if m.bias is not None:
            nn.init.constant(m.bias, 0.0)

def countParameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [ ]:
data_dir = '/content/drive/MyDrive/new_nii_data' # change the path depending on where you store the dataset C:\Users\vinkk\OneDrive\Desktop
image_1 = 'DU01_image.nii'
label_1 = 'DU01_label.nii'
image_2 = 'DU02_image.nii'
label_2 = 'DU02_label.nii' 
image_3 = 'DU03_image.nii' 
label_3 = 'DU03_label.nii' 
image_4 = 'patient1_left_image.nii' 
label_4 = 'patient1_left_label.nii' 
image_5 = 'patient2_left_image.nii' 
label_5 = 'patient2_left_label.nii' 
image_6 = 'patient1_right_image.nii' 
label_6 = 'patient1_right_label.nii'
image_7 = 'patient2_right_image.nii' 
label_7 = 'patient2_right_label.nii'

In [ ]:
image_1 = nib.load(os.path.join(data_dir,image_1)).get_fdata()
label_1 = nib.load(os.path.join(data_dir,label_1)).get_fdata()
image_2 = nib.load(os.path.join(data_dir,image_2)).get_fdata()
label_2 = nib.load(os.path.join(data_dir,label_2)).get_fdata()
image_3 = nib.load(os.path.join(data_dir,image_3)).get_fdata()
label_3 = nib.load(os.path.join(data_dir,label_3)).get_fdata()
image_4 = nib.load(os.path.join(data_dir,image_4)).get_fdata()
label_4 = nib.load(os.path.join(data_dir,label_4)).get_fdata()
image_5 = nib.load(os.path.join(data_dir,image_5)).get_fdata()
label_5 = nib.load(os.path.join(data_dir,label_5)).get_fdata()
image_6 = nib.load(os.path.join(data_dir,image_6)).get_fdata()
label_6 = nib.load(os.path.join(data_dir,label_6)).get_fdata()
image_7 = nib.load(os.path.join(data_dir,image_7)).get_fdata()
label_7 = nib.load(os.path.join(data_dir,label_7)).get_fdata()

In [ ]:
label_1 = (np.array(label_1) > 0).astype(np.int)   #changing the range from 0-255 to 0-1 for labels 
label_2 = (np.array(label_2) > 0).astype(np.int)
label_3 = (np.array(label_3) > 0).astype(np.int)
label_4 = (np.array(label_4) > 0).astype(np.int)
label_5 = (np.array(label_5) > 0).astype(np.int)
label_6 = (np.array(label_6) > 0).astype(np.int)
label_7 = (np.array(label_7) > 0).astype(np.int)

In [ ]:
image_1 = torch.from_numpy(image_1)
image_2 = torch.from_numpy(image_2)
image_3 = torch.from_numpy(image_3)
image_4 = torch.from_numpy(image_4)
image_5 = torch.from_numpy(image_5)
image_6 = torch.from_numpy(image_6)
image_7 = torch.from_numpy(image_7)

label_1 = torch.from_numpy(label_1)
label_2 = torch.from_numpy(label_2)
label_3 = torch.from_numpy(label_3)
label_4 = torch.from_numpy(label_4)
label_5 = torch.from_numpy(label_5)
label_6 = torch.from_numpy(label_6)
label_7 = torch.from_numpy(label_7)

In [ ]:
image_1 = image_1.unsqueeze(0).permute(3,0,1,2)
image_2 = image_2.unsqueeze(0).permute(3,0,1,2)
image_3 = image_3.unsqueeze(0).permute(3,0,1,2)
image_4 = image_4.unsqueeze(0).permute(3,0,1,2)
image_5 = image_5.unsqueeze(0).permute(3,0,1,2)
image_6 = image_6.unsqueeze(0).permute(3,0,1,2)
image_7 = image_7.unsqueeze(0).permute(3,0,1,2)

label_1 = label_1.permute(2,0,1)
label_2 = label_2.permute(2,0,1)
label_3 = label_3.permute(2,0,1)
label_4 = label_4.permute(2,0,1)
label_5 = label_5.permute(2,0,1)
label_6 = label_6.permute(2,0,1)
label_7 = label_7.permute(2,0,1)

In [ ]:
transformed_train_dataset_1 = KneeDataset(image_1, label_1,
                                        transform=(None))
transformed_train_dataset_2 = KneeDataset(image_2, label_2,
                                        transform =(None))
transformed_train_dataset_3 = KneeDataset(image_3, label_3,
                                        transform =(None))
transformed_train_dataset_4 = KneeDataset(image_4, label_4,
                                       transform=(None))
transformed_train_dataset_5 = KneeDataset(image_5, label_5,
                                       transform=(None))
transformed_train_dataset_6 = KneeDataset(image_6, label_6,
                                        transform =(None))
transformed_train_dataset_7 = KneeDataset(image_7, label_7,
                                        transform =(None))
transformed_train_dataset = torch.utils.data.ConcatDataset([transformed_train_dataset_1, transformed_train_dataset_2,transformed_train_dataset_3,
                                                            transformed_train_dataset_4, transformed_train_dataset_5,
                                                            transformed_train_dataset_6,transformed_train_dataset_7])
#transformed_valid_dataset = torch.utils.data.ConcatDataset([transformed_valid_dataset_1, transformed_valid_dataset_2])

image_datasets = {'train':transformed_train_dataset,
                 # 'valid':transformed_valid_dataset,
                  }
dataloaders = {'train' : torch.utils.data.DataLoader(image_datasets['train'], 
                                                        batch_size=20, shuffle = True, num_workers = 4),
                  #'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=20,
                                             #shuffle=True, num_workers=4),
                  
                  }

dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}
print(dataset_sizes)


{'train': 945}


In [ ]:
batch= next(iter(dataloaders['train'])) #train
inputs = batch['image']
label = batch['label']
print(inputs.shape)
print(label.shape)

'''
batch_valid = next(iter(dataloaders['valid'])) #validation
inputs = batch['image']
label = batch['label']
print(inputs.shape)
print(label.shape)
'''

torch.Size([20, 1, 217, 271])
torch.Size([20, 217, 271])


"\nbatch_valid = next(iter(dataloaders['valid'])) #validation\ninputs = batch['image']\nlabel = batch['label']\nprint(inputs.shape)\nprint(label.shape)\n"

In [ ]:
print(torch.unique(batch['label']))

tensor([0, 1])


In [ ]:
criterion = nn.CrossEntropyLoss().cuda() ## Loss
net = UNet_2d()
net.apply(init_weights)
net.cuda()
net.train()
total_epocs_var = 500 #500

print(countParameters(net))

optimizer = optim.Adam(list(net.parameters()),lr=0.002)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  """


31042434


In [ ]:
#from torch.optim.lr_scheduler import StepLR
#scheduler = StepLR(optimizer, step_size=20, gamma=0.1)


In [ ]:
train_loss = torch.zeros(total_epocs_var)/0

dice1 = []  #torch.zeros(500)/0
dice2 = 0
dice3 =[]

for epoch_i in range(total_epocs_var):
    net.train()
    train_loss[epoch_i] = 0.0
    batch_dice_t = 0
    batch_dice_mean_t = 0
    i=0
    
           
      ####MODEL EXPERIMENT####
    for i, batch in enumerate(dataloaders['train']):
    
        optimizer.zero_grad()
        inputs = batch['image'].float().cuda()
        label = batch['label'].long().cuda()
        inputs,label = augmentAffine(inputs,label,0.075)      
        output = net(inputs)
        loss = criterion(output,label)
        loss.backward()
        optimizer.step() 
        train_loss[epoch_i] += loss.item()
        d1 = dice(output.argmax(1), label,2)
        batch_dice_t += d1
        i = i +1
        batch_dice_mean_t += d1.mean().cpu()

    train_loss[epoch_i] /= i
    batch_dice_t/=i
    batch_dice_mean_t/=i
    dice2+=batch_dice_t   
    dice1.append(batch_dice_t)
    dice3.append(batch_dice_mean_t)
        
    if(epoch_i%25==5):
            #print('Run Dice 2 All Labels Average',run_dice2/epoch_i)
            print('epoch',epoch_i,'train loss',train_loss[epoch_i], 'dice train',dice3[epoch_i])
            print('run_dice[epoch] Average',torch.tensor(dice3).float().mean())
            
        
            
print('Run Dice 2 All Labels Average Final',dice2/total_epocs_var)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3448: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


epoch 5 train loss tensor(0.0402) dice train tensor(0.9500)
run_dice[epoch] Average tensor(0.8937)
epoch 30 train loss tensor(0.0172) dice train tensor(0.9760)
run_dice[epoch] Average tensor(0.9523)
epoch 55 train loss tensor(0.0133) dice train tensor(0.9809)
run_dice[epoch] Average tensor(0.9640)
epoch 80 train loss tensor(0.0127) dice train tensor(0.9820)
run_dice[epoch] Average tensor(0.9695)
epoch 105 train loss tensor(0.0186) dice train tensor(0.9744)
run_dice[epoch] Average tensor(0.9719)
epoch 130 train loss tensor(0.0105) dice train tensor(0.9847)
run_dice[epoch] Average tensor(0.9740)
epoch 155 train loss tensor(0.0215) dice train tensor(0.9716)
run_dice[epoch] Average tensor(0.9754)
epoch 180 train loss tensor(0.0094) dice train tensor(0.9862)
run_dice[epoch] Average tensor(0.9767)
epoch 205 train loss tensor(0.0093) dice train tensor(0.9862)
run_dice[epoch] Average tensor(0.9777)
epoch 230 train loss tensor(0.0092) dice train tensor(0.9864)
run_dice[epoch] Average tensor(0.9

In [ ]:
  #Saving the model after training 
torch.save(net.train().cpu(),'/content/drive/MyDrive/training_at_500_inpat_bs20.')
#torch.save(net.eval().cpu().state_dict(),'/content/drive/My Drive/train_300') #state_dict not required 
  # change the path depending on where you want to save in drive 

In [ ]:
# graphs for loss
epoc=range(total_epocs_var)
plt.plot(epoc,train_loss.cpu().numpy(),label='train_loss')
plt.ylim(0,0.05)
plt.legend()
plt.title('Train Loss')
plt.figure()
plt.show()
'''
plt.plot(epoc,val_loss.cpu().numpy(),'r',label='valid_loss')
#plt.ylim(0,0.05)
plt.legend()
plt.title('Valid Loss')
plt.figure()
plt.show()
'''
plt.plot(epoc,dice3,label='Train Dice')
#plt.plot(epoc,val_l,label='Val_loss')
plt.legend()
plt.title('Train Dice Loss')
plt.figure()
plt.show()

'''
plt.plot(epoc,run_dice3,'g',label='Valid Dice')
#plt.plot(epoc,val_l,label='Val_loss')
#plt.ylim(0.3,1)
plt.legend()
plt.title('Valid Dice Loss')
plt.figure()
plt.show()
'''

In [ ]:
 if(True):
        net.eval()
        with torch.no_grad():
            val_loss[epoch_i] = 0.0
            batch_dice = 0
            batch_dice_mean = 0
            i=0
            
            for i, batch in enumerate(dataloaders['valid']):
            
                
                #optimizer.zero_grad()
                inputs_v = batch['image'].float().cuda()
                label_v = batch['label'].long().cuda()
                inputs_v,label_v = augmentAffine(inputs_v,label_v,0.075)
                output_v = net(inputs_v)
                loss = criterion(output_v,label_v)
                val_loss[epoch_i] += loss.item()
              ##  scheduler.step()
                
            
                d1 = dice(output_v.argmax(1), label_v, 2)
                
                batch_dice += d1
                i = i+1
                batch_dice_mean += d1.mean().cpu()
              
            val_loss[epoch_i]/=i
            batch_dice/=i
            batch_dice_mean/=i
            
            run_dice2+=batch_dice
            run_dice.append(batch_dice)
            run_dice3.append(batch_dice_mean)